# PYTHON CHECKER GAME PLAYED WITH SMART COMPUTER AI
## Creator: Lam Tran *(461300)*, Tri Kieu *(464376)*

This purpose of this project/assignment is to create game-playing interface called checker (or draughts) using Pygame package that played with created with a programmed AI computer. AI computer can be made with the method of MinMax algorithms which mostly used in making board game interfaces. The first thing to initialize a constant value for the size of a board, checker pieces, and each square from the board. Addtionally, it is also important to give the checker game interface some color as well using RGB color code.

In [6]:
import pygame

# Initialize the size of Board
WIDTH, HEIGHT = 600,600
ROW, COL = 8,8

# Initialize the size of Square
SQUARE_SIZE = HEIGHT // ROW

# COLOR
BLACK = (0,0,0)
LIGHTBROWN = (221,189,137)
DARKBROWN = (140,93,54)
WHITE = (225,225,225)
GREY = (128, 128, 128)

# Load King
crown = pygame.transform.scale(pygame.image.load('king_master.png'),(40,20))

### Step 1: Creating checker pieces
This class *Piece* will create each and every checker pieces on the board and gives them all the appropriate location of board at the start of the game. It also rewrite the location of the pieces when the player or AI start to make a move. 

In [7]:
class Piece:
    
    #Private variables to draw checker pieces
    PADDING = 8
    OUTLINE = 2
    
    def __init__(self, row, col, color):
        self.row = row
        self.col = col
        self.color = color
        self.isKing = False
        self.x_coord = 0
        self.y_coord = 0
        self.piece_location()

    #Give each pieces a proper location at the start or current state of a game
    def piece_location(self):
        self.x_coord = SQUARE_SIZE * self.col + SQUARE_SIZE // 2
        self.y_coord = SQUARE_SIZE * self.row + SQUARE_SIZE // 2

    # Draw one checker piece when this function is called 
    def draw_piece(self, win):
        radius = SQUARE_SIZE//2 - self.PADDING
        outline = radius + self.OUTLINE
        pygame.draw.circle(win, GREY, (self.x_coord, self.y_coord), outline)
        pygame.draw.circle(win, self.color, (self.x_coord, self.y_coord), radius)

        if self.isKing:
            win.blit(crown,(self.x_coord - crown.get_width() // 2,self.y_coord - crown.get_height() //2))
    
    # Become a king
    def become_king (self):
        self.isKing = True

    # Move a piece
    def move(self,row,col):
        self.row = row
        self.col = col
        self.piece_location()

### Step 2: Creating checker board game
Within the class called *"Board"*, the main program will display the board on application simply by calling the *display_board()* function on *main* class with a required parameter **WINDOW** that had a blank interface for a function *draw_board()* to draw. Additionally, class *Board* data (variable) about the current state of board game, the remaining of normal or king pieces on a board. 

In [8]:
class Board:
    def __init__(self):
        self.board = []
        self.chosen_piece = 0
        self.white_left = self.black_left = 12
        self.white_king = self.black_king = 0
        self.add_pieces_to_board()
        

    # First: Fill the whole board with Dark Brown color.
    # Second: Then the light brown square is placed. It starts from the top left (coordinate (0,0)).
    # cor_x, cor_y shows where to draw the square (x and y coordinate).
    def draw_board(self, win):
        win.fill(DARKBROWN)
        for row in range (ROW):
            for col in range(row % 2, ROW ,2):
                cor_x = row * SQUARE_SIZE
                cor_y = col * SQUARE_SIZE
                pygame.draw.rect(win,LIGHTBROWN, (cor_x,cor_y,SQUARE_SIZE,SQUARE_SIZE))

    # Adding all the checker pieces on the board at specific coordination
    # In terms of specific coordination, it shows how and where the pieces place at the start of a game
    # 0 represent empty space in the board game, whereas object from class Piece will put in that square
    def add_pieces_to_board(self):
        for row in range(ROW):
            self.board.append([])
            for col in range(COL):
                if col % 2 == ((row + 1) % 2):
                    if row < 3:
                        self.board[row].append(Piece(row, col, BLACK))
                    elif row > 4:
                        self.board[row].append(Piece(row, col, WHITE))
                    else:
                        self.board[row].append(0)
                else:
                    self.board[row].append(0)
    
    #Display the board along with other checker pieces at the start of a game
    def display_board(self, win):
        self.draw_board(win)
        for row in range(ROW):
            for col in range(COL):
                checker_piece = self.board[row][col]
                if checker_piece != 0: # If it is not empty (!=0), draw the piece in that coordination
                    checker_piece.draw_piece(win) 

    def move_piece(self,piece,row,col):
        self.board[piece.row][piece.col], self.board[row][col] = self.board[row][col], self.board[piece.row][piece.col]
        piece.move(row, col)

        if row == ROW - 1 or row == 0:
            piece.make_king()
            if piece.color == WHITE:
                self.white_kings += 1
            else:
                self.red_kings += 1 


    def get_pos(self,row,col):
        return self.board[row][col]

### Step 3: Mouse Class Handling
This will receive the mouse position

In [9]:
def mouse_pos_cal_mouse(pos):
    x, y = pos
    row = y // SQUARE_SIZE
    col = x // SQUARE_SIZE
    return row,col
    

### MAIN CLASS PROGRAM
This main class is where to start the checker game after running all the classes above.  

In [10]:
class main:
    
    status = True
    board = Board()
    FPS = 60

    WINDOW = pygame.display.set_mode((WIDTH,HEIGHT))
    pygame.display.set_caption("Checkers")

    clock = pygame.time.Clock()

    while status == True:
        clock.tick(FPS)

        for action in pygame.event.get():
            if action.type == pygame.QUIT:
                status = False          
            if action.type == pygame.MOUSEBUTTONDOWN:
                place = pygame.mouse.get_pos()
                row,col = mouse_pos_cal_mouse(place)
                piece = board.get_pos(row,col)
                board.move_piece(piece, 5,6)


        board.display_board(WINDOW)
        pygame.display.update()
    
    pygame.quit()



AttributeError: 'int' object has no attribute 'row'